In [68]:
import pandas as pd
import numpy as np
import json
import urllib.request
import sqlite3

In [79]:
# Copied from anki connect site
def request(action: str, **params) -> dict:
    return {"action": action, "params": params, "version": 6}


# Copied from anki connect site
def invoke(action: str, **params):
    requestJson = json.dumps(request(action, **params)).encode("utf-8")
    response = json.load(
        urllib.request.urlopen(
            urllib.request.Request("http://127.0.0.1:8765", requestJson)
        )
    )
    if len(response) != 2:
        raise Exception("response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("response is missing required error field")
    if "result" not in response:
        raise Exception("response is missing required result field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]

In [87]:
db_path = "../data/clean/bir_warehouse.db"
query: str = """
SELECT y.year_name, ex.exam_type, q.question, qo.option_text, qo.is_correct
FROM questions_options AS qo
JOIN questions AS q ON qo.question_id = q.id
JOIN year AS y ON q.exam_year = y.id_year
JOIN exam AS ex ON q.exam_subject = ex.id_type;
"""

con = sqlite3.connect(db_path)

exams_df: pd.DataFrame = pd.read_sql(query,con)

con.close()

exams_df["index"] = exams_df["year_name"] + exams_df["exam_type"] + exams_df["question"]
exams_df["year_name"] = exams_df["year_name"].astype(int)
exams_df["is_correct"] = exams_df["is_correct"].astype(bool)

exams_df.head()

,year_name,exam_type,question,option_text,is_correct,index
0,2024,bir,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,False,2024bir1. La barrera hematoencefálica:
1,2024,bir,1. La barrera hematoencefálica:,2. Es permeable al O2 y al CO2.,True,2024bir1. La barrera hematoencefálica:
2,2024,bir,1. La barrera hematoencefálica:,3. Es impermeable al etanol.,False,2024bir1. La barrera hematoencefálica:
3,2024,bir,1. La barrera hematoencefálica:,4. Es impermeable al agua.,False,2024bir1. La barrera hematoencefálica:
4,2024,bir,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,False,2024bir2. El espacio subaracnoideo se encuentra:


In [88]:
exams_df_group = exams_df.groupby("index")
exams_df_group.head()

,year_name,exam_type,question,option_text,is_correct,index
0,2024,bir,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,False,2024bir1. La barrera hematoencefálica:
1,2024,bir,1. La barrera hematoencefálica:,2. Es permeable al O2 y al CO2.,True,2024bir1. La barrera hematoencefálica:
2,2024,bir,1. La barrera hematoencefálica:,3. Es impermeable al etanol.,False,2024bir1. La barrera hematoencefálica:
3,2024,bir,1. La barrera hematoencefálica:,4. Es impermeable al agua.,False,2024bir1. La barrera hematoencefálica:
4,2024,bir,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,False,2024bir2. El espacio subaracnoideo se encuentra:
...,...,...,...,...,...,...
24276,2013,bir,235. En la electroforesis de hemoglobina en ac...,1. Se emplean como colorantes el Rojo Neutro o...,False,2013bir235. En la electroforesis de hemoglobin...
24277,2013,bir,235. En la electroforesis de hemoglobina en ac...,2. Se obtienen tres fraccione s diferenciadas:...,True,2013bir235. En la electroforesis de hemoglobin...
24278,2013,bir,235. En la electroforesis de hemoglobina en ac...,3. La fracción mayoritaria es la HbA2.,False,2013bir235. En la electroforesis de hemoglobin...
24279,2013,bir,235. En la electroforesis de hemoglobina en ac...,4. Se obtiene el hemolizado empleando CINH4.,False,2013bir235. En la electroforesis de hemoglobin...


In [89]:
col_names = ["year", "exam", "question", "option1", "option2", "option3", "option4", "option5", "correct"]
exams_df_pivoted = pd.DataFrame(columns= col_names)

In [90]:
n_question: int = 0
quest_dict = {}
for _, group in exams_df_group:
    question: list = [group["year_name"].iloc[0], group["exam_type"].iloc[0], group["question"].iloc[0]]
    for row_tuple in group.itertuples():
        question.append(row_tuple[4])
    if group["year_name"].iloc[0] > 2014:
        question.append("blank")
    correct = ([i+1 for i, boolean in enumerate(group["is_correct"]) if boolean] or [0])[0]
    question.append(correct)
    if len(question) > 9:
        question.append("0")
    exams_df_pivoted.loc[n_question] = [str(x) for x in question]
    n_question += 1

In [91]:
exams_df_pivoted.head()

,year,exam,question,option1,option2,option3,option4,option5,correct
0,2005,bir,1. Un agente que está impidiendo la división b...,1. Antibiótico.,2. Bactericida.,3. Bacteriostático.,4. Desinfectante.,5. Esterilizante.,3
1,2005,bir,10. Los transposones bacterianos:,1. Poseen replicación autónoma.,2. Son un tipo de propfago.,3. No tienen secuencias de inserción.,4. Son plásmidos especializados.,5. Pueden incluir genes de resistencia a anti...,5
2,2005,bir,100. La fibronectina es uno de los componentes...,1. La matriz extracelular.,2. El citoesqueleto.,3. La lámina nuclear.,4. Las uniones estrechas.,5. Los complejos de poro nucleares.,1
3,2005,bir,"101. En una fibra nerviosa mielínica, la condu...",1. Con decremento.,2. Saltatoria.,3. Más lenta que en una fibra amielínica.,4. De tipo electrotónico.,5. Más lenta cuanto más gruesa sea la fibra.,2
4,2005,bir,102. Los potenciales de acción se dan:,1. Cuando la membrana se hiperpolariza.,2. Tras un aumento transitorio de la permeabil...,3. Por una disminución de la permeabilidad al ...,4. Con una amplitud variable según la fuerza d...,5. Sólo en las neuronas.,2


In [95]:
exams_df_clean = exams_df_pivoted

In [96]:
exams_df_clean["year"] = exams_df_clean["year"].astype(int)
exams_df_clean["correct"] = exams_df_clean["correct"].astype(int)
exams_df_clean["num_q"] = exams_df_clean["question"].apply(lambda x: x.split(".", 1)[0])
exams_df_clean["num_q"] = exams_df_clean["num_q"].astype(int)
exams_df_clean["option5"] = exams_df_clean["option5"].replace("blank", np.nan)
exams_df_clean.head()

,year,exam,question,option1,option2,option3,option4,option5,correct,num_q
0,2005,bir,1. Un agente que está impidiendo la división b...,1. Antibiótico.,2. Bactericida.,3. Bacteriostático.,4. Desinfectante.,5. Esterilizante.,3,1
1,2005,bir,10. Los transposones bacterianos:,1. Poseen replicación autónoma.,2. Son un tipo de propfago.,3. No tienen secuencias de inserción.,4. Son plásmidos especializados.,5. Pueden incluir genes de resistencia a anti...,5,10
2,2005,bir,100. La fibronectina es uno de los componentes...,1. La matriz extracelular.,2. El citoesqueleto.,3. La lámina nuclear.,4. Las uniones estrechas.,5. Los complejos de poro nucleares.,1,100
3,2005,bir,"101. En una fibra nerviosa mielínica, la condu...",1. Con decremento.,2. Saltatoria.,3. Más lenta que en una fibra amielínica.,4. De tipo electrotónico.,5. Más lenta cuanto más gruesa sea la fibra.,2,101
4,2005,bir,102. Los potenciales de acción se dan:,1. Cuando la membrana se hiperpolariza.,2. Tras un aumento transitorio de la permeabil...,3. Por una disminución de la permeabilidad al ...,4. Con una amplitud variable según la fuerza d...,5. Sólo en las neuronas.,2,102


In [97]:
validate_option_1 = (exams_df_clean["option1"].str.startswith("1")).all()
validate_option_2 = (exams_df_clean["option2"].str.startswith("2")).all()
validate_option_3 = (exams_df_clean["option3"].str.startswith("3")).all()
validate_option_4 = (exams_df_clean["option4"].str.startswith("4")).all()
validate_option_5 = (exams_df_clean["option5"][exams_df_clean["option5"].notna()].str.startswith("5")).all()

if not all([validate_option_1, validate_option_2, validate_option_3, validate_option_4, validate_option_5]):
    raise Warning
print("DataFrame validated")

DataFrame validated
